# PortADa NLP Workshop
# Notebook 03: LLMs
## Author: Juan Manuel Pérez, 2024


In this notebook we will use ChatGPT in different settings. Our main goal is to parse text from OCRed data in order to get it tabulated.



In [1]:
!pip install openai tiktoken


[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
# Get API key from getpass
from getpass import getpass

# Set the API key just for this session

api_key = getpass("Enter your API key: ")

In [3]:
import openai

openai.api_key = api_key

## First example

Let's try to use ChatGPT to detect whether a passage of text contains a departure or arrival of a ship in a port. 

In [5]:
passages = [
    "De Valencia y Tarragona en 5 d. laud Sto. Cristo, de 30 t., piloto D. J. F. Adam, con 45 sacos arroz, 30 de habichuelas y 50 cahices salvado á D. J. Estrany, 40 sacos arroz á Don J. Fontanillas, 35 cargas loza y 33 serones azulejos á D. R. Girona.",
    "La inflación en España se situó en el 2,7% en el mes de julio, lo que supone una décima más que en junio, según el indicador adelantado del índice de precios de consumo (IPC) publicado este jueves por el Instituto Nacional de Estadística (INE).",
    "El Puerto de Buenos Aires es el principal puerto de contenedores de Buenos Aires y de Argentina, concentrando aproximadamente un 60 % del movimiento de contenedores del país. Es también uno de los puertos más importantes de la región latinoamericana debido a su moderna infraestructura y capacidad de carga.",

    "Proveniente de Coruña, Cádiz y Cartagena en 21 d. polacra-goleta Segunda Clavellina, de 85 t., c. D. J. Ventura Perez, con 30 pipas sardina á D. C. Rabella, 15 id. á D. F. Raola, 8 id. á D. B. Fiol, 7 id. á los Sres. Soler y Esteve, 6 id. á D. D. Robert, 5 id., 3 de anchoa y 53 de congrio à D. J. Serra y Totosaus, 28 cajas vidrios á D. L. Grau, 6 id. á D. R. Girona, 1 id. á D. A. Basti, 4 cascos caparrosa á la Sra. viuda Rosiñol, 6 barriles crémor á D. G. Caña- dó, 4 1/2 pipas parocha y anchoa á D. B. Solá y Amat, y 8 fardos congrio á D. S. Seler.",

    "De Rosario es Lionel Messi, el mejor jugador de fútbol del mundo. Nacido en Rosario, Argentina, el 24 de junio de 1987, Messi es considerado uno de los mejores jugadores de fútbol de todos los tiempos. Ha ganado numerosos premios y títulos, incluyendo ocho Balones de Oro",
]

We will detect/classify if the given passage contains a departure or arrival of a ship in a port. To do this, we will use so called "prompting", particularly in the zero-shot mode.

What is zero-shot? Zero-shot learning is a way of training models to perform tasks without having to provide examples of the task. 



In [8]:
template_prompt = """Given the following passage, answer 'YES' or 'NO' if the passage is about an arrival or departure of a ship from a port.
###
Passage: {}
Answer:
"""

for passage in passages:
    prompt = template_prompt.format(passage)
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[{
            "role": "system",
            "content": prompt
        }]
    )
    print("="*80)
    print("Passage: ", passage)
    print("Answer: ", response.choices[0].message.content)

Passage:  De Valencia y Tarragona en 5 d. laud Sto. Cristo, de 30 t., piloto D. J. F. Adam, con 45 sacos arroz, 30 de habichuelas y 50 cahices salvado á D. J. Estrany, 40 sacos arroz á Don J. Fontanillas, 35 cargas loza y 33 serones azulejos á D. R. Girona.
Answer:  YES
Passage:  La inflación en España se situó en el 2,7% en el mes de julio, lo que supone una décima más que en junio, según el indicador adelantado del índice de precios de consumo (IPC) publicado este jueves por el Instituto Nacional de Estadística (INE).
Answer:  NO
Passage:  El Puerto de Buenos Aires es el principal puerto de contenedores de Buenos Aires y de Argentina, concentrando aproximadamente un 60 % del movimiento de contenedores del país. Es también uno de los puertos más importantes de la región latinoamericana debido a su moderna infraestructura y capacidad de carga.
Answer:  NO
Passage:  Proveniente de Coruña, Cádiz y Cartagena en 21 d. polacra-goleta Segunda Clavellina, de 85 t., c. D. J. Ventura Perez, con

In [7]:
response.choices

[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='YES', role='assistant', function_call=None, tool_calls=None))]

## Tabulating OCRed data

We will use ChatGPT to parse text from OCRed data in order to get it tabulated. We will use the following passage of text as an example:


In [9]:
text= """De Valencia y Tarragona en 5 d. laud Sto. Cristo, de 30 t., piloto D. J. F. Adam, con 45 sacos arroz, 30 de habichuelas y 50 cahices salvado á D. J. Estrany, 40 sacos arroz á Don J. Fontanillas, 35 cargas loza y 33 serones azulejos á D. R. Girona.

De Buen, Marin, Cádiz y Salou en 25 d. polacra-goleta Diligencia, de 94 t., p. C. Ra mon Martinez, con 44 1/2 pipas sardina á D. G. Dotras, 28 1/2 id. á D. R. Tomás y C.ª, 1 id. á D. J. Serra y Totosaus, 10 1/2 id. á D. I. Moreu, 7 1/2 á D. J. Poch y 7 id. á D. J Pou.


De la Coruña, Cádiz y Cartagena en 21 d. polacra-goleta Segunda Clavellina, de 85 t., c. D. J. Ventura Perez, con 30 pipas sardina á D. C. Rabella, 15 id. á D. F. Raola, 8 id. á D. B. Fiol, 7 id. á los Sres. Soler y Esteve, 6 id. á D. D. Robert, 5 id., 3 de anchoa y 53 de congrio à D. J. Serra y Totosaus, 28 cajas vidrios á D. L. Grau, 6 id. á D. R. Girona, 1 id. á D. A. Basti, 4 cascos caparrosa á la Sra. viuda Rosiñol, 6 barriles crémor á D. G. Caña- dó, 4 1/2 pipas parocha y anchoa á D. B. Solá y Amat, y 8 fardos congrio á D. S. Seler.

De Charleston en 42 d. polacra goleta Salud, de 130 t., c. D. J. Magri, con 293 balas
algodon á D. B. Fiol.-Ha dado declaracion á las doce del dia.


De Marsella en 24 horas vapor Guadalquivir, de 158 t., c. D. G. Villaverde, con 200,000 francos á los señores Vidal y Cuadras hermanos, 115,000 id. á los señores Girona hermanos. Clavé y Compañía, 52,200 id. á D. J. Torrens, 40,000 id. á D. I. Villavecchia, 40,000 id. á D. J. M. Serra, 25,142 id. á D. M. Roig y Rom, 25,000 id. á D. R. Vinent, 25,000 á los señores Dotres, Clavé y Fabra, 25,000 id. á los señores Plandolit hermanos, 15,000 id. á D. J. Martí Codolar, 10,000 id. á los señores Stagno, Torrens y Compañía, 6,000 id. á D. T. Ray- nal, laneria, sedería, algodonería, quincalla, drogas, papel pintado, sombreros, maquina- ria, relojería, perfumería, porcelana y otros efectos para esta y 157 bultos de varios géneros de tránsito y 26 pasajeros, consignado á los señores Martorell y Bofill.-Ha tenido entrada á las dos y media de la tarde.


De Noya, Muros y Salou en 28 d. goleta Luisita, de 82 t., c. D. F. Fernandez Fajardo, con 56 cascos sardina á D. J. Poch, 61 id. á D. J. Novell, 37 id. á los Sres. Romeu, Tomás y compañia, y 8 id. á D. G. Dotres.

De Newport en 28 d. bergantin Nomade, de 124 t., c. F. Janffret, con 140 toneladas carbon de piedra á los Sres. Serra hermanos.-Queda entredicho.

Además 3 buques de la costa de este Principado, con 119 pipas vino trashordo, 22 qq. corteza de pino á D. J. Lombrich, y leña.


Polacra española Esperanza, c. D. A. Lloret, para Cartagena con alquitran y drogas de tránsito.-Laud Trinidad, p. V. Cevero, para Cullera en lastre.-Id. S. Sebastian, p. F. Muelas, para id. en id.-Id. S. Sebastian, p. F. Juan, para id. en id.-Id. San Francisco, p. J. B. Martinez, para Vinaroz con aros de hierro, otros efectos y lastre.-Id. Rosario, p. H. Fábregas, para id. en lastre.-Id. Caridad, p. C. Subirat, para Alicante en id.-Id. Ven- tura, p. S. Alexadre, para Valencia en id.-Id. Lealtad, p. V. Jover, para id. con 20 balas de algodon.-Id. Tridente, p. P. Sans, para id. con 50 buitos géneros.-Id. Carmen, p. M. Terrasa, para id. en id.-Id. Rival, c. J. Blake, para Zante en id.-Ademas 20 buques para la costa de este Principado con efectos y lastre.
"""


First, let's try to use ChatGPT to perform a easy NER task.

In [11]:

# Create a completion

prompt = f"""Detectar ciudades mencionadas en el texto. Devolver una lista con - por cada ciudad encontrada.
{text}
"""

response = openai.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[{
        "role": "system",
        "content": prompt
    }],
)

In [12]:

print(response.choices[0].message.content)

- Valencia
- Tarragona
- Cádiz
- Salou
- La Coruña
- Cartagena
- Charleston
- Marsella
- Noya
- Muros
- Newport
- Cartagena
- Cullera
- San Francisco
- Vinaroz
- Alicante
- Lealtad
- Tridente
- Carmen
- Zante


We can ask to get it in JSON format!

In [14]:

# Create a completion

prompt = f"""Detectar ciudades mencionadas en el texto. Devolver un json con el campo 'ciudades'
{text}
"""

response = openai.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[{
        "role": "system",
        "content": prompt
    }],
    response_format={"type": "json_object"}
)

print(response.choices[0].message.content)

{
    "ciudades": [
        "Valencia",
        "Tarragona",
        "Buen",
        "Marin",
        "Cádiz",
        "Salou",
        "la Coruña",
        "Cartagena",
        "Charleston",
        "Marsella",
        "Noya",
        "Muros",
        "Newport",
        "Cullera",
        "Vinaroz",
        "Alicante",
        "Zante"
    ]
}


In [15]:
### Parsing each departure

In [17]:
template_prompt = """Para cada línea de texto, detectar si se trata de un envío particular de una ciudad a otra. Devolver un json con los siguientes campos:

- origen: ciudad de origen (list of string )
- capitán: nombre del capitán (string)
- barco: nombre del barco (string)
- carga: descripción de la carga (list of strings)
- fecha: fecha del envío (string)

En caso de que no se pueda detectar alguno de los campos, devolver null. En caso de que la línea no sea un envío, devolver null.

###
{line}
"""


for line in text.split("\n"):
    line = line.strip()

    if not line:
        continue


    prompt = template_prompt.format(line=line)
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[{
            "role": "system",
            "content": prompt
        }],
        response_format={"type": "json_object"}
    )
    print("="*80)
    print(response.choices[0].message.content)


{
  "origen": ["Valencia", "Tarragona"],
  "capitán": "D. J. F. Adam",
  "barco": "Sto. Cristo",
  "carga": ["45 sacos arroz", "30 de habichuelas", "50 cahices salvado", "40 sacos arroz", "35 cargas loza", "33 serones azulejos"],
  "fecha": "5 d."
}
{
  "origen": ["Buen", "Marin", "Cádiz", "Salou"],
  "capitán": "C. Ramon Martinez",
  "barco": "Diligencia",
  "carga": ["44 1/2 pipas sardina a D. G. Dotras", "28 1/2 id. a D. R. Tomás y C.", "1 id. a D. J. Serra y Totosaus", "10 1/2 id. a D. I. Moreu", "7 1/2 a D. J. Poch", "7 id. a D. J Pou"],
  "fecha": "25 d."
}
{
  "origen": [
    "La Coruña",
    "Cádiz",
    "Cartagena"
  ],
  "capitán": "D. J. Ventura Perez",
  "barco": "Segunda Clavellina",
  "carga": [
    "30 pipas sardina á D. C. Rabella",
    "15 id. á D. F. Raola",
    "8 id. á D. B. Fiol",
    "7 id. á los Sres. Soler y Esteve",
    "6 id. á D. D. Robert",
    "5 id.",
    "3 de anchoa",
    "53 de congrio à D. J. Serra y Totosaus",
    "28 cajas vidrios á D. L. Grau",
    

In [32]:

print(response.choices[0].message.content)


{
    "origen": "Valencia",
    "destino": "Tarragona",
    "capitán": "D. J. F. Adam",
    "barco": "Sto. Cristo",
    "carga": [
        "45 sacos arroz",
        "30 de habichuelas",
        "50 cahices salvado",
        "40 sacos arroz",
        "35 cargas loza",
        "33 serones azulejos"
    ],
    "fecha": "5 d."
}


## Exercise:


Check out the [`data/ocred`](https://github.com/finiteautomata/portada_nlp/tree/main/data/ocred) directory in the repository and try to parse it